In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [8]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold,train_test_split
import json

In [3]:
img = nib.load('../data/Task09_Spleen/imagesTr/spleen_10.nii.gz')
img = np.array(img.dataobj)

In [4]:
img.shape

(512, 512, 55)

In [5]:
patients = [i for i in os.listdir('../data/Task09_Spleen/imagesTr/') if i.find('._sp')==-1]

In [6]:
patients

['spleen_18.nii.gz',
 'spleen_63.nii.gz',
 'spleen_16.nii.gz',
 'spleen_41.nii.gz',
 'spleen_62.nii.gz',
 'spleen_6.nii.gz',
 'spleen_47.nii.gz',
 'spleen_61.nii.gz',
 'spleen_60.nii.gz',
 'spleen_59.nii.gz',
 'spleen_22.nii.gz',
 'spleen_28.nii.gz',
 'spleen_25.nii.gz',
 'spleen_14.nii.gz',
 'spleen_3.nii.gz',
 'spleen_17.nii.gz',
 'spleen_52.nii.gz',
 'spleen_49.nii.gz',
 'spleen_56.nii.gz',
 'spleen_24.nii.gz',
 'spleen_40.nii.gz',
 'spleen_13.nii.gz',
 'spleen_45.nii.gz',
 'spleen_9.nii.gz',
 'spleen_26.nii.gz',
 'spleen_46.nii.gz',
 'spleen_38.nii.gz',
 'spleen_21.nii.gz',
 'spleen_32.nii.gz',
 'spleen_53.nii.gz',
 'spleen_12.nii.gz',
 'spleen_19.nii.gz',
 'spleen_20.nii.gz',
 'spleen_27.nii.gz',
 'spleen_29.nii.gz',
 'spleen_8.nii.gz',
 'spleen_44.nii.gz',
 'spleen_33.nii.gz',
 'spleen_31.nii.gz',
 'spleen_2.nii.gz',
 'spleen_10.nii.gz']

In [7]:
patients[0][:-7]

'spleen_18'

In [28]:
names = []

os.makedirs('../data/Task09_Spleen/processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'../data/Task02_Heart/imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'../data/Task02_Heart/labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'../data/Task02_Heart/processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i].reshape(320, 320,1)
        temp_seg = seg[:,:,i].reshape(320, 320,1)
        
        
        np.save(f'../data/Task02_Heart/processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'../data/Task02_Heart/processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [11]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [13]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    images_train = []
    for patient in patients_train:
        images_train+=[
                f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
    
    
    images_val = []
    for patient in patients_test:
        images_val+=[
                f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
                for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
                if i.find('.npy') != -1 and i.find('seg') == -1
            ]
        
    
    split = {
        'train': images_train,
        'val': images_val,
    }
    
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)

In [14]:
patients

array(['spleen_18.nii.gz', 'spleen_63.nii.gz', 'spleen_16.nii.gz',
       'spleen_41.nii.gz', 'spleen_62.nii.gz', 'spleen_6.nii.gz',
       'spleen_47.nii.gz', 'spleen_61.nii.gz', 'spleen_60.nii.gz',
       'spleen_59.nii.gz', 'spleen_22.nii.gz', 'spleen_28.nii.gz',
       'spleen_25.nii.gz', 'spleen_14.nii.gz', 'spleen_3.nii.gz',
       'spleen_17.nii.gz', 'spleen_52.nii.gz', 'spleen_49.nii.gz',
       'spleen_56.nii.gz', 'spleen_24.nii.gz', 'spleen_40.nii.gz',
       'spleen_13.nii.gz', 'spleen_45.nii.gz', 'spleen_9.nii.gz',
       'spleen_26.nii.gz', 'spleen_46.nii.gz', 'spleen_38.nii.gz',
       'spleen_21.nii.gz', 'spleen_32.nii.gz', 'spleen_53.nii.gz',
       'spleen_12.nii.gz', 'spleen_19.nii.gz', 'spleen_20.nii.gz',
       'spleen_27.nii.gz', 'spleen_29.nii.gz', 'spleen_8.nii.gz',
       'spleen_44.nii.gz', 'spleen_33.nii.gz', 'spleen_31.nii.gz',
       'spleen_2.nii.gz', 'spleen_10.nii.gz'], dtype='<U16')

In [9]:
patients = np.array(patients)

patients_train, patients_test = train_test_split(patients, test_size=0.2, random_state=42)

patients_train, patients_val = train_test_split(patients_train, test_size=0.25, random_state=42)

a
images_train = []
for patient in patients_train:
    images_train+=[
            f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
            for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
            if i.find('.npy') != -1 and i.find('seg') == -1
        ]


images_val = []
for patient in patients_test:
    images_val+=[
            f'../data/Task09_Spleen/processed_data/{patient}/'+i[:-4]
            for i in sorted(os.listdir(f'../data/Task09_Spleen/processed_data/{patient}/'))
            if i.find('.npy') != -1 and i.find('seg') == -1
        ]


split = {
    'train': images_train,
    'val': images_val,
}


with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
        json.dump(split, outfile)

NameError: name 'a' is not defined

In [13]:
len(patients_train)

41

In [12]:
len(patients_val)

8

In [19]:
f'../data/Task02_Heart/processed_data/{patients_train}/'

"../data/Task02_Heart/processed_data/['la_007' 'la_019' 'la_009' 'la_016' 'la_004' 'la_022' 'la_018' 'la_014'\n 'la_011' 'la_010' 'la_024' 'la_020' 'la_023' 'la_030' 'la_021' 'la_017']/"